# Manage prompt size

Agents dynamically call tools, and the results of those tools get add back to the prompt so that the agent can plan the next action. Depending on what tools are being used and how they're being called, the agent prompt could grow larger than the model context window.

With LCEL, it's easy to add custom functionality for managing the size of prompts. Here's a simple example of how we can do this. In this example we'll simply drop the earliest function messages when we bump up against the model's context window limit.

In [1]:
# !pip install langchain duckduckgo-search

from operator import itemgetter

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.chat import ChatPromptValue
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import format_tool_to_openai_function

In [2]:
tools = [DuckDuckGoSearchRun().with_retry(stop_after_attempt=3)]
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
def condense_prompt(prompt: ChatPromptValue) -> ChatPromptValue:
    messages = prompt.to_messages()
    num_tokens = llm.get_num_tokens_from_messages(messages)
    function_messages = messages[2:]
    while num_tokens > 4_000:
        function_messages = function_messages[1:]
        num_tokens = llm.get_num_tokens_from_messages(messages[:2] + function_messages)
    messages = messages[:2] + function_messages
    return ChatPromptValue(messages=messages)


agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | condense_prompt
    | llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {"input": "Who is the current US president? What's their home state? What's their home state's bird? What's that bird's scientific name?"}
)

KeyError: 'tools'

In [4]:
getattr(tools[0], "args_schema")

langchain.tools.ddg_search.tool.DDGInput

In [11]:
tools[0].args_schema

langchain.tools.ddg_search.tool.DDGInput